In [1]:
from keras_vggface.vggface import VGGFace

In [2]:
import keras

In [3]:
import tensorflow as tf

In [4]:
vggface = VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3), pooling='avg')

In [8]:
!pip install opencv-python

  Using cached opencv_python-4.5.1.48-cp38-cp38-win_amd64.whl (34.9 MB)


In [40]:
!pip install matplotlib

  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached kiwisolver-1.3.1-cp38-cp38-win_amd64.whl (51 kB)


In [41]:
!pip install Pillow

In [47]:
!pip install mtcnn

  Using cached mtcnn-0.1.0-py3-none-any.whl (2.3 MB)


In [48]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN

In [53]:
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = plt.imread(filename)
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    face1 = Image.fromarray(face)
    face1.save("myface.jpg")
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array
 
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
    # extract faces
    faces = [extract_face(f) for f in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create a vggface model
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat
 
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
    else:
        print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))
 
# define filenames
filenames = ['1.jpg','2.jpg']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
# define sharon stone
sharon_id = embeddings[0]
# verify known photos of sharon
print('Positive Tests')
is_match(embeddings[0], embeddings[1])
# verify known photos of other people
print('Negative Tests')
is_match(embeddings[0], embeddings[1])

Positive Tests
>face is a Match (0.264 <= 0.500)
Negative Tests
>face is a Match (0.264 <= 0.500)
